In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()


pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

%matplotlib inline

from matplotlib import pyplot as plt
import matplotlib.ticker as mtick # For specifying the axes tick format 

import seaborn as sns
import re

import json, os, gc, math, time
import datetime
import collections
from tqdm import tqdm
import glob

from statistics import mean
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

from sklearn import metrics

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [3]:
#ドライブへのアクセス
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/00_datascience/19_ufj_bank"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/00_datascience/19_ufj_bank


In [4]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sub = pd.read_csv('input/sample_submit.csv')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9791 entries, 0 to 9790
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            9791 non-null   object
 1   goal          9791 non-null   object
 2   country       9791 non-null   object
 3   duration      9791 non-null   int64 
 4   category1     9791 non-null   object
 5   category2     9791 non-null   object
 6   html_content  9791 non-null   object
 7   state         9791 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 612.1+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            9800 non-null   object
 1   goal          9800 non-null   object
 2   country       9800 non-null   object
 3   duration      9800 non-null   int64 
 4   category1     9800 non-null   object
 5   category2     9800 non-null   object
 6   html_content  9800 non-null   object
dtypes: int64(1), object(6)
memory usage: 536.1+ KB


In [7]:
train.head()

,id,goal,country,duration,category1,category2,html_content,state
0,train_00000,20001-21000,US,45,art,mixed media,"<div class=""contents""><div><p><a href=""http://dummy.com"">http://dummy.com<p>In its first year, The Shillito's Elves Display won an international \ndesign award for Shillito's department store. The elves display is arts\n and crafts at its finest. The mixed media exhibit displays the talents\n of local fine arts graduates, and the display, while ""folksy"", is as \ntechnologically advanced as Disney World's famous ""It's a Small World"" \nride. </p><p>The Shillito's Elves attracted close to 100...",1
1,train_00001,19001-20000,US,59,food,restaurants,"<div class=""contents""><div><p>Cultural Pretzel Sports Bar is a place where people can come and watch their favorite local and world wide sports teams, while enjoying their favorite ethnic foods on a soft pretzel. Our menu includes a variety of appetizers and soft pretzels. Our pretzels include - Mexican Taco, Mexican Steak Fajita, Greek Gyro, Italian Sausage and Peppers, and American Steak Philly Soft Pretzels. With more to be added to the menu as the business grows.</p><p>Cultural Pretze...",0
2,train_00002,2001-3000,US,38,art,performance art,"<div class=""contents""><div><p>I want to perform this piece guerilla style, off the back off a 24 Ft truck on Christmas Eve, amid last minute Christmas shopping in New York City.</p>\n<p>Our truck pulls in, the door rolls up and I perform. </p>\n<p>New York is a city where you just do it - no excuses. A place where the new becomes the normal real fast and where the artist is challenged to capture the attention and the hearts of the unshockable and unstoppable. In times of celebration and re...",0
3,train_00003,1001-2000,US,30,art,mixed media,"<div class=""contents""><div><div class=""template asset"" contenteditable=""false"" data-alt-text="""" data-caption="""" data-id=""_xxx_"">\n<figure>\n<img alt="""" class=""fit lazyload"" data-src=""http://dummy.com""/>\n</figure>\n</div>\n<h1 class=""page-anchor"" id=""_xxx_"">\n<figure>\n<img alt=""Canyon de Chelley, Dine' (Navajo) Reservation, Arizona. Photo by: Demian Dine' Yazhi'"" class=""fit lazyload"" data-src=""http://dummy.com""/>\n<figcaption class=""px2"">Canyon de Chelley, Dine' (Navajo) Reservation, Arizo...",1
4,train_00004,1001-2000,US,29,film & video,webseries,"<div class=""contents""><div><p>The story of the show, both on and off screen, is about daring to dream after something impossible.Even with an incredible amount of difficulty and opposition, the project continued to grow in size and depth. A year ago, we were finally able to release the pilot episode of Azusa Ghost Hunter's Society (check it out! <a href=""http://dummy.com"">\n<figure>\n<img alt="""" class=""fit lazyload"" data-src=""http://dummy.com""/>\n</figure>\n</div>\n<p>Which brings us to now...",1


# bertの設定

In [5]:
#htmlタグのクリーニング
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [6]:
#htmlタグのクリーニングの実行
for i in range(len(train)):
  train.loc[i, 'cleaned_text'] = remove_html(train.loc[i, 'html_content'])

for i in range(len(test)):
  test.loc[i, 'cleaned_text'] = remove_html(test.loc[i, 'html_content'])

In [7]:
#bertによる特徴抽出
import torch
import transformers

from transformers import BertTokenizer


class BertSequenceVectorizer:
    def __init__(self, model_name="bert-base-uncased", max_len=256):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_name = model_name
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = max_len

    def vectorize(self, sentence: str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out.cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out.detach().numpy()

In [8]:
BSV = BertSequenceVectorizer()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# train_dataのcsv出力

In [12]:
seq_out_train = train['cleaned_text'].progress_apply(lambda x: BSV.vectorize(x))


  0%|          | 0/9791 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors


In [13]:
#ベースのデータフレーム作る
context_mean_f = np.mean(seq_out_train[0], axis=0)
batch_mean_f = np.mean(context_mean_f, axis=0)
seq_train_df = pd.DataFrame(batch_mean_f).T

In [14]:
#最終層のデータを768次元に落とす
for i in range(1, len(train)):
  context_mean = np.mean(seq_out_train[i], axis=0)
  batch_mean = np.mean(context_mean, axis=0)
  seq_train_df = seq_train_df.append(pd.DataFrame(batch_mean).T)

In [15]:
seq_train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.129976,0.309543,0.489337,-0.098177,0.282953,-0.150489,-0.008439,0.428571,-0.134610,-0.294867,0.050036,-0.501957,0.004306,0.610372,-0.056495,0.457846,0.283214,-0.014021,-0.003365,0.301193,0.233740,-0.165883,0.167558,0.558328,0.322316,0.154834,-0.083820,-0.210163,-0.189294,-0.115295,0.455405,0.008023,-0.051641,-0.339214,-0.042465,-0.333371,-0.104562,-0.265214,-0.049338,0.205207,-0.348619,-0.202791,0.133455,-0.308525,-0.262340,-0.255759,0.079052,-0.078591,0.011927,-0.015285,-0.492733,0.269305,-0.207360,0.028627,0.231964,0.374693,-0.332890,-0.222865,-0.371295,-0.392889,0.080282,-0.090564,0.107396,-0.472089,-0.130044,0.022151,0.325466,0.452082,-0.383168,-0.078437,-0.039586,-0.022349,0.035902,-0.084724,-0.016193,0.002300,-0.237222,0.238447,0.149575,-0.039879,-0.217432,0.274231,0.034600,0.594440,0.246495,0.020685,-0.023747,0.284438,-0.350095,0.587324,-0.031158,-0.342824,0.163217,0.113896,0.341550,-0.265933,0.112192,-0.144232,-0.261847,0.384266,0.189921,-0.172670,0.134168,0.065880,-0.040974,-0.005879,0.140794,0.339912,0.000908,0.292106,0.297149,-0.098030,-0.440458,-0.217403,0.179887,-0.015950,0.272276,-0.053358,-0.106783,0.140299,0.330490,-0.086973,0.062124,0.317206,0.010380,-0.075034,-0.279116,0.281662,-0.090237,-0.008078,0.211430,0.186265,0.093695,-0.339976,-0.009571,0.062832,-0.164317,-0.417851,-0.216700,0.225317,-0.003030,-0.204273,0.437598,0.200266,0.132737,-0.197711,0.117923,-0.037461,-0.126198,0.274727,0.098990,-0.012672,-0.330414,-0.230924,0.037412,-0.153292,-0.575341,0.165937,-0.051080,-0.005319,0.410425,-0.100394,-0.173134,0.119826,-0.084918,0.113210,-0.033582,0.439082,-0.357555,0.035787,0.076930,0.097488,0.650917,0.195011,0.041463,0.095911,0.408281,0.085721,0.091184,-0.043678,-0.447519,0.408591,-0.150677,0.019453,0.167728,-0.156639,0.267482,-0.373730,0.110632,-0.277527,-0.446491,-0.078024,-0.260692,0.022304,0.338321,0.141939,-0.163644,-0.036051,-0.289672,-0.070195,0.073211,0.097493,0.080082,0.169067,-0.145146,-0.488731,-0.030744,-0.150814,-0.100048,-0.157825,-0.128318,0.231783,0.047307,0.396067,-0.008358,-0.067914,0.159022,-0.173628,-0.081158,-0.174492,-0.15968

In [16]:
seq_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 0
Columns: 768 entries, 0 to 767
dtypes: float32(768)
memory usage: 28.8 MB


In [17]:
seq_train_df.to_csv('input/02_seq_train_df.csv')

# test dataのcsv出力

In [9]:
seq_out_test = test['cleaned_text'].progress_apply(lambda x: BSV.vectorize(x))

  0%|          | 0/9800 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (624 > 512). Running this sequence through the model will result in indexing errors


In [12]:
#ベースのデータフレーム作る
context_mean_f = np.mean(seq_out_test[0], axis=0)
batch_mean_f = np.mean(context_mean_f, axis=0)
seq_test_df = pd.DataFrame(batch_mean_f).T

In [13]:
#最終層のデータを768次元に落とす
for i in range(1, len(test)):
  context_mean = np.mean(seq_out_test[i], axis=0)
  batch_mean = np.mean(context_mean, axis=0)
  seq_test_df = seq_test_df.append(pd.DataFrame(batch_mean).T)

In [14]:
seq_test_df.to_csv('input/02_seq_test_df.csv')